In [2]:
import pandas as pd
import os
import openai
from dotenv import load_dotenv, find_dotenv

In [3]:
load_dotenv(find_dotenv())
openai.api_key = os.environ.get("openai_api")

In [7]:
df = pd.read_csv("../data/raw/SentimentLabeled_10112022.csv")
df["text"].head()

0    I commend all of law enforcement for the secur...
1    Socialist tyrant Maduro starves, tortures his ...
2    Socialist tyrant Maduro starves, tortures his ...
3    Inhofe: The Canadians have already had convers...
4    Statement on Iran nuclear agreement. http://t....
Name: text, dtype: object

In [27]:
def isRelevant(tweet):

    prompt = """
    You are a machine that can only answer "True" or "False". For example, for the following question:
    Q: "Is Joe Biden the President of the United States"
    The answer would be:
    A: True

    With this in mind, determine the validity of this statement: The following Tweet is relevant to China
    TWEET: \n"""

    prompt += tweet + "\n" + "ANSWER: \n\n"
    
    response = openai.Completion.create(engine = "text-davinci-003", prompt = prompt)
    answer = response["choices"][0]["text"].strip()
    
    return answer

def cleanAnswer(response):
   
    try:
        return eval(response)
    except:
        if "False" in response:
            return False
        elif "True" in response:
            return True
        else:
            return "Error"
            

In [10]:
subset = df.sample(n = 100)
subset = subset[["text", "Bucket"]]
subset["Relevant"] = subset["Bucket"].apply(lambda x: x == "1")
subset.drop(columns = ["Bucket"], inplace = True)
subset.head()

,text,Relevant
4747,In abandoning the Iran Nuclear Agreement and h...,False
14272,Yet another reversal of Trump's efforts to pro...,False
7342,"@NYCHA For our neighbors who speak Chinese, pl...",False
6335,China controls too much of America’s supply-ch...,True
13686,I wrote an op-ed in the @dcexaminer this week ...,True


In [13]:
subset["gpt_Relevant"] = subset["text"].apply(isRelevant)

In [28]:
subset["gpt_Relevant"] = subset["gpt_Relevant"].apply(cleanAnswer)

In [29]:
subset["gpt_Relevant"].value_counts()

True     58
False    42
Name: gpt_Relevant, dtype: int64

In [30]:
(subset["Relevant"] == subset["gpt_Relevant"]).mean()

0.65

In [31]:
subset.head(20)

,text,Relevant,gpt_Relevant
4747,In abandoning the Iran Nuclear Agreement and h...,False,False
14272,Yet another reversal of Trump's efforts to pro...,False,False
7342,"@NYCHA For our neighbors who speak Chinese, pl...",False,False
6335,China controls too much of America’s supply-ch...,True,True
13686,I wrote an op-ed in the @dcexaminer this week ...,True,True
1716,Fentanyl is responsible for more than 40% of t...,True,False
4580,(2/3) It seems the President is finding that e...,False,False
6160,"China is the source of many viral outbreaks, b...",True,True
6462,#ICYMI (2018): America’s national security res...,True,True
8208,#Crapo4YearHighlight: The Fiscal Year 2020 NDA...,True,True
